<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/kursPawe%C5%82Krakowiak/neural-network-course/03_keras/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 50000 recenzji filmów oznaczonych sentymentem: pozytywny/negatywny. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

### <a name='a1'></a> 1. Import bibliotek

In [1]:
# ######################################################
# Etap 7. Odcinek: Główne problemy uczenia głębokiego - przeuczenie i niedouczenie.

%tensorflow_version 2.x
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

'2.4.1'

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [2]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [3]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [4]:
# pierwsza recenzja
# Na przykład druga liczba 14 oznacza słowo, które pojawiło się na 14 miejscu
# pod względem częstości (w całym zbiorze wszystkich recenzji)
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
# ta funkcja mapuje każdą liczbę całkowitą na słowo
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2  # unknown nieznany, prawdopodobnie wypadło z listy 10000 pierwszych słowek
word_to_idx['<UNUSED>'] = 3  # unused nieużywany
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))
# to nasza recenzja odpowiadająca liście train_data[0]
# Właśnie drugie słowo to 'film', które jest na 14 miejscu pod względem częstości

1646592/1641221 [==============================] - 0s 0us/step
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole fi

In [6]:
# klasa 1 - pozytywna
# klasa 0 - negatywna
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [8]:
# funkcja wrzucająca dane do sieci. Kodowanie 0-1. Tworzymy macierz o wymiarze
# liczba sekwencji na wymiar naszego kodowania. 
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

# dimension to liczba przekazanych słów czyli wcześniejsze NUM_WORDS=10000
train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
# 25000 próbek i 10000 słów
train_data.shape
# każda próbka ma wstawić 0 lub 1 - w zależności czy zawiera słowo znajdujące się 
# wśród 10000 najczęstszych słów.

(25000, 10000)

In [9]:
test_data.shape

(25000, 10000)

### <a name='a3'></a> 3. Budowa modelu bazowego

In [10]:
# ######################################################
# Etap 7. Odcinek: Główne problemy uczenia głębokiego - przeuczenie i niedouczenie cz. 2.

baseline_model = Sequential()
# warstwą wejściową jest 10000 słów dobranych ze zbioru danych
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
# model klasyfikacji binarnej więc 1 neuron (albo recenzja pozytywna albo negatywna)
# funkcja sigmoid zwraca p-two
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                160016    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [11]:
# trenowanie modelu
# Rozmiar danych wejściowych batch_size=512 po którym następuje aktualizacja wag
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))
# na zbiorze treningowym accuracy: 1.0000 więc model się przeuczył

Epoch 1/20
49/49 [==============================] - 4s 62ms/step - loss: 0.5901 - accuracy: 0.7259 - binary_crossentropy: 0.5901 - val_loss: 0.3412 - val_accuracy: 0.8758 - val_binary_crossentropy: 0.3412
Epoch 2/20
49/49 [==============================] - 2s 40ms/step - loss: 0.2667 - accuracy: 0.9091 - binary_crossentropy: 0.2667 - val_loss: 0.2837 - val_accuracy: 0.8880 - val_binary_crossentropy: 0.2837
Epoch 3/20
49/49 [==============================] - 2s 39ms/step - loss: 0.1871 - accuracy: 0.9380 - binary_crossentropy: 0.1871 - val_loss: 0.2957 - val_accuracy: 0.8807 - val_binary_crossentropy: 0.2957
Epoch 4/20
49/49 [==============================] - 2s 39ms/step - loss: 0.1471 - accuracy: 0.9498 - binary_crossentropy: 0.1471 - val_loss: 0.3055 - val_accuracy: 0.8816 - val_binary_crossentropy: 0.3055
Epoch 5/20
49/49 [==============================] - 2s 38ms/step - loss: 0.1200 - accuracy: 0.9638 - binary_crossentropy: 0.1200 - val_loss: 0.3330 - val_accuracy: 0.8761 - val_bin

### <a name='a4'></a> 3. Budowa 'mniejszego' modelu

In [12]:
# różni się tym że ma mniej neuronów w warstwach ukrytych
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 40004     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [13]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 55ms/step - loss: 0.6704 - accuracy: 0.5674 - binary_crossentropy: 0.6704 - val_loss: 0.5634 - val_accuracy: 0.8388 - val_binary_crossentropy: 0.5634
Epoch 2/20
49/49 [==============================] - 2s 37ms/step - loss: 0.5074 - accuracy: 0.8569 - binary_crossentropy: 0.5074 - val_loss: 0.4010 - val_accuracy: 0.8688 - val_binary_crossentropy: 0.4010
Epoch 3/20
49/49 [==============================] - 2s 41ms/step - loss: 0.3363 - accuracy: 0.8994 - binary_crossentropy: 0.3363 - val_loss: 0.3205 - val_accuracy: 0.8822 - val_binary_crossentropy: 0.3205
Epoch 4/20
49/49 [==============================] - 2s 44ms/step - loss: 0.2466 - accuracy: 0.9233 - binary_crossentropy: 0.2466 - val_loss: 0.2941 - val_accuracy: 0.8882 - val_binary_crossentropy: 0.2941
Epoch 5/20
49/49 [==============================] - 2s 39ms/step - loss: 0.2057 - accuracy: 0.9351 - binary_crossentropy: 0.2057 - val_loss: 0.2871 - val_accuracy: 0.8876 - val_bin

### <a name='a5'></a> 4. Budowa 'większego' modelu

In [14]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               5120512   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [15]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 16s 325ms/step - loss: 0.4308 - accuracy: 0.7919 - binary_crossentropy: 0.4308 - val_loss: 0.2917 - val_accuracy: 0.8816 - val_binary_crossentropy: 0.2917
Epoch 2/20
49/49 [==============================] - 15s 308ms/step - loss: 0.1423 - accuracy: 0.9525 - binary_crossentropy: 0.1423 - val_loss: 0.3371 - val_accuracy: 0.8751 - val_binary_crossentropy: 0.3371
Epoch 3/20
49/49 [==============================] - 15s 312ms/step - loss: 0.0453 - accuracy: 0.9888 - binary_crossentropy: 0.0453 - val_loss: 0.4664 - val_accuracy: 0.8654 - val_binary_crossentropy: 0.4664
Epoch 4/20
49/49 [==============================] - 15s 309ms/step - loss: 0.0082 - accuracy: 0.9991 - binary_crossentropy: 0.0082 - val_loss: 0.5838 - val_accuracy: 0.8691 - val_binary_crossentropy: 0.5838
Epoch 5/20
49/49 [==============================] - 15s 311ms/step - loss: 0.0011 - accuracy: 1.0000 - binary_crossentropy: 0.0011 - val_loss: 0.6874 - val_accuracy: 0.8664

In [16]:
# atrybut history pozwala pokazać wszystkie trenowane parametry
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.491047,0.80908,0.491047,0.341198,0.87580,0.341198,0
1,0.253614,0.91112,0.253614,0.283733,0.88796,0.283733,1
2,0.185782,0.93540,0.185782,0.295697,0.88072,0.295697,2
3,0.149800,0.94856,0.149800,0.305526,0.88164,0.305526,3
4,0.123908,0.95968,0.123908,0.332999,0.87608,0.332999,4


### <a name='a6'></a> 5. Porównanie wydajności modeli

In [17]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()
# model baseline:
# zielona to funkcja straty na zbiorze treningowym (model ma zadanie ją minimalizować)
# fioletowa to funkcja straty na zbiorze walidacyjnym już od 3 epoki zaczyna rosnąć
# czyli model zaczyna się przeuczać bo gorzej sobie radzi
# na zbiorze walidacyjnym (na danych których nie widział), czyli overfitting.
# Dlatego warto zakończyć trenowanie po 3 epoce.

# model smaller:
# ma większą stratę na zbiorze treningowym niż baseline bo ma mniejszą architekturę.
# Ale na walidacyjnym też ma mniejszą stratę - zaczyna ona rosnąć po 4 epoce czyli wtedy
# warto skończyć trenowanie

# bigger:
# model bardzo szybko się przeuczył na zbiorze treningowym. Strata walidacyjna od razu zaczęła
# rosnąć. Dlatego niedobry model

# Wybierać model który ma najmniejszą stratę walidacyjną. Czyli wybrać smaller

### <a name='a7'></a> 6. Metody regularyzacji

In [18]:
# ######################################################
# Etap 7. Odcinek: Metody regularyzacji modeli.

# Jednym ze sposobów zapobiegania przeuczania jest dodanie warstwy Dropout,
# która porzuca część neuronów w warstwach ukrytych.

# Inną techniką jest dodanie funkcji regulatora do macierzy wag jądra poprzez 
# parametr kernel_regularizer

from tensorflow.keras.regularizers import l2

l2_model = Sequential()
# w kernel_regularizer wybrana metryka l2
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [19]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 4s 75ms/step - loss: 0.7954 - accuracy: 0.6883 - binary_crossentropy: 0.6050 - val_loss: 0.5303 - val_accuracy: 0.8723 - val_binary_crossentropy: 0.3658
Epoch 2/20
49/49 [==============================] - 2s 41ms/step - loss: 0.4587 - accuracy: 0.9019 - binary_crossentropy: 0.2971 - val_loss: 0.4447 - val_accuracy: 0.8869 - val_binary_crossentropy: 0.2928
Epoch 3/20
49/49 [==============================] - 2s 41ms/step - loss: 0.3645 - accuracy: 0.9290 - binary_crossentropy: 0.2157 - val_loss: 0.4214 - val_accuracy: 0.8863 - val_binary_crossentropy: 0.2830
Epoch 4/20
49/49 [==============================] - 2s 40ms/step - loss: 0.3206 - accuracy: 0.9396 - binary_crossentropy: 0.1848 - val_loss: 0.4135 - val_accuracy: 0.8844 - val_binary_crossentropy: 0.2856
Epoch 5/20
49/49 [==============================] - 2s 41ms/step - loss: 0.2909 - accuracy: 0.9478 - binary_crossentropy: 0.1652 - val_loss: 0.4085 - val_accuracy: 0.8834 - val_bin

In [20]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

# w modelu regularyzowanym l2_model_history strata na zbiorze walidacyjnym (fioletowy) 
# jest mniejsza niż na odpowiadającym mu zbiorze nieregularyzowanym (wcześniejszym - pomarańczowy)

In [21]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [22]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 55ms/step - loss: 0.6661 - accuracy: 0.5764 - binary_crossentropy: 0.6661 - val_loss: 0.5302 - val_accuracy: 0.8276 - val_binary_crossentropy: 0.5302
Epoch 2/20
49/49 [==============================] - 2s 37ms/step - loss: 0.5143 - accuracy: 0.7801 - binary_crossentropy: 0.5143 - val_loss: 0.3908 - val_accuracy: 0.8775 - val_binary_crossentropy: 0.3908
Epoch 3/20
49/49 [==============================] - 2s 37ms/step - loss: 0.3948 - accuracy: 0.8586 - binary_crossentropy: 0.3948 - val_loss: 0.3111 - val_accuracy: 0.8866 - val_binary_crossentropy: 0.3111
Epoch 4/20
49/49 [==============================] - 2s 38ms/step - loss: 0.3267 - accuracy: 0.8890 - binary_crossentropy: 0.3267 - val_loss: 0.2949 - val_accuracy: 0.8872 - val_binary_crossentropy: 0.2949
Epoch 5/20
49/49 [==============================] - 2s 37ms/step - loss: 0.2749 - accuracy: 0.9102 - binary_crossentropy: 0.2749 - val_loss: 0.2863 - val_accuracy: 0.8876 - val_bin

In [23]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [24]:
# sam: łączę obie techniki regularyzacji

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

l2_and_dropout_model = Sequential()
# w kernel_regularizer wybrana metryka l2
l2_and_dropout_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
l2_and_dropout_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
dropout_model.add(Dropout(0.5))
l2_and_dropout_model.add(Dense(1, activation='sigmoid'))

l2_and_dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_and_dropout_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                160016    
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [25]:
l2_and_dropout_history = l2_and_dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 48ms/step - loss: 0.7802 - accuracy: 0.6858 - binary_crossentropy: 0.5935 - val_loss: 0.5187 - val_accuracy: 0.8737 - val_binary_crossentropy: 0.3561
Epoch 2/20
49/49 [==============================] - 2s 39ms/step - loss: 0.4454 - accuracy: 0.9066 - binary_crossentropy: 0.2860 - val_loss: 0.4420 - val_accuracy: 0.8867 - val_binary_crossentropy: 0.2940
Epoch 3/20
49/49 [==============================] - 2s 38ms/step - loss: 0.3570 - accuracy: 0.9296 - binary_crossentropy: 0.2126 - val_loss: 0.4185 - val_accuracy: 0.8860 - val_binary_crossentropy: 0.2850
Epoch 4/20
49/49 [==============================] - 2s 37ms/step - loss: 0.3112 - accuracy: 0.9415 - binary_crossentropy: 0.1806 - val_loss: 0.4111 - val_accuracy: 0.8827 - val_binary_crossentropy: 0.2890
Epoch 5/20
49/49 [==============================] - 2s 38ms/step - loss: 0.2869 - accuracy: 0.9470 - binary_crossentropy: 0.1672 - val_loss: 0.4057 - val_accuracy: 0.8814 - val_bin

In [26]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2_and_dropout'], [baseline_history, l2_and_dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()